# Learning PyTorch: Custom nn Modules

Source: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#pytorch-custom-nn-modules

A third order polynomial, trained to predict `y=sin(x)` from `−π` to `pi` by minimizing squared Euclidean distance.

This implementation defines the model as a custom Module subclass. Whenever you want a model more complex than a simple sequence of existing Modules you will need to define your model this way.

We will use a problem of fitting `y=sin(x)` with a third order polynomial as our running example. The network will have four parameters, and will be trained with gradient descent to fit random data by minimizing the Euclidean distance between the network output and the true output.

The `forward` function computes output Tensors from input Tensors. The `backward` function receives the gradient of the output Tensors with respect to some scalar value, and computes the gradient of the input Tensors with respect to that same scalar value.

If `x` is a Tensor that has `x.requires_grad=True` then `x.grad` is another Tensor holding the gradient of `x` with respect to some scalar value.

In [1]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. 
        We can use Modules defined in the constructor as well as arbitrary operators on Tensors.
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'

In [2]:
# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

In [3]:
# Both x and y will have torch.Size([2000]).
x.size(), y.size()

(torch.Size([2000]), torch.Size([2000]))

In [4]:
# Construct our model by instantiating the class defined above.
model = Polynomial3()

In [5]:
# Construct our loss function and an Optimizer. 
# The call to model.parameters() in the SGD constructor will contain the 
# learnable parameters (defined with torch.nn.Parameter) which are members of the model.

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = criterion(y_pred, y)
    if t % 100 == 99: # print every 100 mini-batches
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 386.1968688964844
199 274.6533508300781
299 196.1319580078125
399 140.8386993408203
499 101.89057922363281
599 74.4481201171875
699 55.107215881347656
799 41.472694396972656
899 31.858680725097656
999 25.078086853027344
1099 20.294830322265625
1199 16.91991424560547
1299 14.538225173950195
1399 12.85718822479248
1499 11.670477867126465
1599 10.832606315612793
1699 10.240942001342773
1799 9.823080062866211
1899 9.527929306030273
1999 9.319430351257324
Result: y = 0.023481333628296852 + 0.8536451458930969 x + -0.004050920717418194 x^2 + -0.09289004653692245 x^3
